In [57]:
import sys
import requests
import pandas as pd
import xlsxwriter
from json import loads
from datetime import date

### Function for Transforming DataFrame in Excel Object

In [16]:
#Function for Writing .xlsx Data Tables which will provide a standard output for table formatting 
def xlsx_table_writer(data_table,sheet_name,col_width_list,title_str,startrow,startcol,worksheet,workbook,writer):

    #Layout/Formatting
    t_vert_spacing = 5 #Vertical Spacing between tables
    t_horz_spacing = 1 #Horizontal Spacing between tables
    title_format = workbook.add_format({'bold': True, 'font_size' : 20, 'fg_color' : '#76933C', 'font_color' : 'white' }) #Standardised Title Format for all tables
    header_format = workbook.add_format({'bold' : True, 'font_size' : 12, 'text_wrap' : True}) #text-wrapping for table headers
    
    # Adding and changing active sheet
    try:
        worksheet=workbook.add_worksheet(sheet_name)
        writer.sheets[sheet_name] = worksheet 
        worksheet.set_zoom(70)
        startrow = 3
        startcol = 1
    except:
        pass

    for i in range(len(col_width_list)):
        worksheet.set_column(i+startcol, i+startcol, col_width_list[i])

    #Writing in Cell Data and Merging Cells for Table Titles
    data_table.to_excel(writer,sheet_name=sheet_name,startrow=startrow , startcol=startcol, index=False, header=False)
    worksheet.merge_range(startrow-2, startcol,startrow-2,data_table.shape[1] + startcol -1, title_str,title_format) #writing in title formatting above table

    #Column settings to use in add table function
    column_settings = [{'header' : column} for column in data_table.columns]

    #Populating Excel with Table Format - Adding table to xls for each df
    worksheet.add_table(startrow-1, startcol, startrow + data_table.shape[0]-1, data_table.shape[1] + startcol - 1, {'columns' : column_settings, 'style': 'Table Style Medium 4', 'autofilter' : False})   

    #Applying a text wrap to the Column Header
    for col_num, value in enumerate(data_table.columns.values):
        worksheet.write(startrow-1, col_num + startcol, value, header_format)
    
    #Setting Positions of Following Tables Insertions
    startrow = startrow + data_table.shape[0] + t_vert_spacing #Setting to start row for next table
    # startrow = 3  
    # startcol = startcol + data_table.shape[1] + t_horz_spacing  #Disabling horizontally displaced tables in favour of vertically displacements
    startcol = 1

    #Setting the column width at the end of the table to keep to spacing minimal between tables
    worksheet.set_column(startcol-1,startcol-1,1)
    
    #Return the start row in order to index for future function calls
    return [startrow,startcol,worksheet,workbook,writer]

In [5]:
#Function to Detect Operating System and Adjust Pathing to Respective Filesystem
def pathing(folder_path,filename):

    #Windows Operating System
    if 'win' in sys.platform:
        if folder_path == 'root':
            filepath = f'{sys.path[0]}\\{filename}'
        else:
            filepath = f'{sys.path[0]}\\{folder_path}\\{filename}'
    #Linux/Mac Operating Sytem
    else:
        if folder_path == 'root':
            filepath = f'{sys.path[0]}/{filename}'
        else:
            filepath = f'{sys.path[0]}/{folder_path}/{filename}'

    return filepath

### User Inputs

In [6]:
user = "Filpill"

### User Overview

In [76]:
url = f"https://www.codewars.com/api/v1/users/{user}"
response = requests.get(url)
json_data = loads(response.text)
print("Status:",response.status_code)
curated_cols = ['username','honor','leaderboardPosition','ranks.overall.name','ranks.languages.python.score','ranks.languages.sql.score','ranks.overall.score','codeChallenges.totalCompleted']
df_profile = pd.json_normalize(json_data)
profile_cols = list(df_profile.columns)
df_profile = df_profile[df_profile.columns.intersection(curated_cols)]
df_profile

Status: 200


,username,honor,leaderboardPosition,ranks.overall.name,ranks.overall.score,ranks.languages.python.score,ranks.languages.sql.score,codeChallenges.totalCompleted
0,Filpill,1016,23238,4 kyu,1405,923,482,86


### List of Completed Challenges

In [65]:
page = 0
url = f"https://www.codewars.com/api/v1/users/{user}/code-challenges/completed?page={page}"
response = requests.get(url)
json_data = loads(response.text)
print("Status:",response.status_code)
raw_data = pd.json_normalize(json_data)
df_complete = pd.DataFrame.from_dict(raw_data.loc[0,'data'])
df_complete.drop(columns = 'slug',inplace=True)
for i,dt in enumerate(df_complete['completedAt']): df_complete.loc[i,'completedAt'] = dt[0:10] #Fixing Date Formats, Only extracting date component
df_complete['completedAt'] = pd.to_datetime(df_complete['completedAt'],format='%Y-%m-%d')
df_complete['Completion Period'] = df_complete['completedAt'].dt.strftime("%b-%y")
print(type(df_complete['completedAt'][0]))
completed_list = list(df_complete['id'])
df_complete.head(2)

Status: 200
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


,id,name,completedLanguages,completedAt,Completion Period
0,5324945e2ece5e1f32000370,Sum Strings as Numbers,[python],2022-11-11,Nov-22
1,521c2db8ddc89b9b7a0000c1,Snail,[python],2022-11-09,Nov-22


### Getting Details of Code Challenges Completed By User

In [9]:
url = f"https://www.codewars.com/api/v1/code-challenges/{completed_list[0]}"
response = requests.get(url)
json_data = loads(response.text)
details_cols =  list(pd.json_normalize(json_data).columns)
df_kata = pd.DataFrame(columns=details_cols) #Making empty dataframe with relevant columns

#Looping through all the completed challenge id's
for kata_id in completed_list:
    url = f"https://www.codewars.com/api/v1/code-challenges/{kata_id}"
    response = requests.get(url)
    json_data = loads(response.text)
    row = pd.json_normalize(json_data)
    df_kata = pd.concat([df_kata,row]) 
df_kata.head(2)

,id,name,slug,category,publishedAt,approvedAt,languages,url,createdAt,description,...,contributorsWanted,rank.id,rank.name,rank.color,createdBy.username,createdBy.url,approvedBy.username,approvedBy.url,unresolved.issues,unresolved.suggestions
0,5324945e2ece5e1f32000370,Sum Strings as Numbers,sum-strings-as-numbers,algorithms,2014-03-15T17:58:42.203Z,2014-03-31T19:33:53.949Z,"[javascript, csharp, php, cpp, c, java, lua, p...",https://www.codewars.com/kata/5324945e2ece5e1f...,2014-03-15T17:56:46.451Z,Given the string representations of two intege...,...,False,-4,4 kyu,blue,nicknovitski,https://www.codewars.com/users/nicknovitski,jhoffner,https://www.codewars.com/users/jhoffner,14,6
0,521c2db8ddc89b9b7a0000c1,Snail,snail,algorithms,2013-08-28T04:56:02.580Z,None,"[ruby, coffeescript, javascript, python, haske...",https://www.codewars.com/kata/521c2db8ddc89b9b...,2013-08-27T04:40:24.711Z,"## Snail Sort\n\nGiven an `n x n` array, retur...",...,True,-4,4 kyu,blue,stevenbarragan,https://www.codewars.com/users/stevenbarragan,NaN,NaN,6,6


### Splitting Out The Details Into Lists

In [77]:
challenge_details = list(df_kata.columns)
selected_details = ['id','totalAttempts','totalCompleted','rank.name','rank.color','category','tags']
print(challenge_details)
df_details_select = df_kata[df_kata.columns.intersection(selected_details)]
df_details_select.head(2)

['id', 'name', 'slug', 'category', 'publishedAt', 'approvedAt', 'languages', 'url', 'createdAt', 'description', 'totalAttempts', 'totalCompleted', 'totalStars', 'voteScore', 'tags', 'contributorsWanted', 'rank.id', 'rank.name', 'rank.color', 'createdBy.username', 'createdBy.url', 'approvedBy.username', 'approvedBy.url', 'unresolved.issues', 'unresolved.suggestions']


,id,category,totalAttempts,totalCompleted,tags,rank.name,rank.color
0,5324945e2ece5e1f32000370,algorithms,490630,28848,"[Strings, Big Integers, Algorithms]",4 kyu,blue
0,521c2db8ddc89b9b7a0000c1,algorithms,401079,53787,"[Arrays, Algorithms]",4 kyu,blue


### Merging Details onto Completitons via the Kata ID

In [82]:
df_kata_combined = pd.merge(df_complete,df_details_select,on='id',how='left')
df_kata_combined.head(2)

,id,name,completedLanguages,completedAt,Completion Period,category,totalAttempts,totalCompleted,tags,rank.name,rank.color
0,5324945e2ece5e1f32000370,Sum Strings as Numbers,[python],2022-11-11,Nov-22,algorithms,490630,28848,"[Strings, Big Integers, Algorithms]",4 kyu,blue
1,521c2db8ddc89b9b7a0000c1,Snail,[python],2022-11-09,Nov-22,algorithms,401079,53787,"[Arrays, Algorithms]",4 kyu,blue


### Re-naming Columns

In [101]:
df_profile.rename(columns={ 'username':'Username',
                           'honor':'Honor',
                           'leaderboardPosition':'Leaderboard',
                           'ranks.overall.name':'Overall Rank',
                           'ranks.languages.python.score':'Python Score',
                           'ranks.languages.sql.score':'SQL Score',
                           'ranks.overall.score':'Total Score'
                           },inplace=True)

df_kata_combined.rename(columns={'id':'Kata ID',
                           'name':'Kata Name',
                           'completedLanguages':'Languages',
                           'completedAt':'Completed Date',
                           'category':'Category',
                           'totalAttempts':'Total Attempts',
                           'totalCompleted':'Total Completed',
                           'tags':'Tags',
                           'rank.name':'Rank',
                           'rank.color':'Color'
                           },inplace=True)

### Initialising XlsxWriter Workbook And Exporting Tables to .xlsx

In [116]:
#-------------------------------------------------------
#-------------.xlsx workbook initialisation-------------
#-------------------------------------------------------

#Excel Output Filepath
excelpath = pathing('xlsx_data',f'codewars_stats_{user}.xlsx')

#Generating .xlsx workbook in which to record dataframes for usage.
date_str = date.today().strftime("%d-%b-%Y")
writer = pd.ExcelWriter(excelpath,engine='xlsxwriter')   
workbook=writer.book
worksheet='' #Initialise variable
startrow = 0 #Initialise variable
startcol = 0 #Initialise variable

In [117]:
#Summary Table
data_table = df_profile
sheet_name = 'Stats'
title_str = f'{user} Codewars Stats since {date_str}'
col_width_list = [31,60,22,26,15,15,17,11,60,11,11]
[startrow,startcol,worksheet,workbook,writer] = xlsx_table_writer(data_table,sheet_name,col_width_list,title_str,startrow,startcol,worksheet,workbook,writer)

In [118]:
#Summary Table
data_table = df_kata_combined
sheet_name = 'Stats'
title_str = f'Kata Compeleted By {user} since {date_str}'
[startrow,startcol,worksheet,workbook,writer] = xlsx_table_writer(data_table,sheet_name,col_width_list,title_str,startrow,startcol,worksheet,workbook,writer)

In [119]:
#Save Excel File Close workbook instance
writer.close()